# Complete Neuron

## Perameters

- `learning_rate`
- `activation`
- `loss`
- `optimizer`

In [52]:
#
# Imports
#

import numpy as np
from collections.abc import Iterable
from numbers import Number

In [53]:
#
# Customer Exceptions
#

class InvalidInitialWeights(Exception): pass

class UnexpectedInputsShape(Exception): pass
class UnexpectedOutputsShape(Exception): pass

class InvalidActivation(Exception): pass
class InvalidOptimizer(Exception): pass
class InvalidLoss(Exception): pass

In [54]:
#
# Sigmoid Activation
#

class Sigmoid():

  def apply(self, x):
    return 1 / (1 + np.exp(-x))
  
  def apply_derivative(self, x):
    sig = self.apply(x)
    return sig * (1 - sig)

In [55]:
#
# Mean Squared Error
#

class MeanSquaredError():
  def calculate(self, y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

In [56]:
#
# Stochastic Gradient Descent
#

class StochasticGradientDescent():
  def __init__(self, learning_rate=None):
    self.learning_rate = learning_rate

  def update(self, model, learning_rate=0.01):
    # learning rate
    lr = self.learning_rate or model.learning_rate or learning_rate

    # hierarchy functions
    def update_neuron(neuron, lr): neuron.update(lr)
    def update_layer(layer, lr):
      for neuron in layer.neurons: update_neuron(neuron, lr)

    # if ann
    if hasattr(model, 'layers'):
      for layer in model.layers: update_layer(layer, lr)
    
    # if layer
    elif hasattr(model, 'neurons'):
      for neuron in model.neurons: update_neuron(neuron, lr)
    
    # if neuron
    elif hasattr(model, 'forward'):
      update_neuron(model, lr)



In [84]:
class Neuron():

  # 
  # Props
  # 

  builtin:dict = {
    'activation': {
      'default': Sigmoid,
      'sigmoid': Sigmoid,
    },
    'loss': {
      'default': MeanSquaredError,
      'sigmoid': MeanSquaredError,
    },
    'optimizer': {
      'default': StochasticGradientDescent,
      'sgd': StochasticGradientDescent,
    }
  }

  errors:dict = {
    'activation': InvalidActivation,
    'loss':       InvalidLoss,
    'optimizer':  InvalidOptimizer,
  }

  input:Iterable = []
  output:float = 0.0

  delta:float = 0.0


  weight_gradient:Iterable = []
  bias_gradient:float = 0.0



  # 
  # Constructor
  # 

  def __init__(
      self,

      input_size:int,
      
      learning_rate:float=0.1,
      activation:str|object='default',
      loss:str|object='default',
      optimizer:str|object='default',

      initial_weights:Number|Iterable=0.1,
      initial_bias:Number=0
    ):

    #> input size and learning rate

    # Input Size
    self.input_size = input_size
    self.input = np.repeat(0, self.input_size)
    
    # Learning Rate
    self.learning_rate = learning_rate

    #> activation, loss and optimizer

    params = {
      'activation': activation,
      'loss':       loss,
      'optimizer':  optimizer,
    }

    # iterate all
    for idx, target in enumerate(['activation', 'loss', 'optimizer']):

      # get param
      param = params[target]
      
      # if object, set directly
      if type(param) == object: setattr(self, target, param)

      # if present in builtin, set
      elif type(param) == str and param in self.builtin[target].keys():
        setattr(self, target, self.builtin[target][param]())

      # raise error
      # if not in {object, str} or string and not in builtin
      else: raise self.errors[target](param)


    #> Initial Weights & Bias

    # initial weights
    if isinstance(initial_weights, Number):                                            self.weights = np.repeat(initial_weights, input_size).astype(float)
    elif isinstance(initial_weights, Iterable) and len(initial_weights) == input_size: self.weights = np.array(initial_weights).astype(float)
    else: raise InvalidInitialWeights(f'Value must be either a number or an Iterable of size {input_size}')

    # initialize gradient to zero
    self.weight_gradient = np.repeat(0.0, self.input_size)

    # initial bias
    self.bias = initial_bias


  
  #
  # Forward
  #

  def forward(self, input):
    self.input = np.array(input)

    # if 1d, predict
    if len(self.input.shape) and self.input.shape[0] == self.input_size:
      # weighted sum
      weighted_sum = np.dot(self.input, self.weights) + self.bias

      # activation
      predicted = self.activation.apply(weighted_sum)

      # return
      return predicted
    
    # elif 2d, predict multiple
    return self.forward_multiple(self.input)



  #
  # Forward Multiple samples
  #

  def forward_multiple(self, inputs):
    inputs = np.array(inputs)

    if len(inputs.shape) < 2 or inputs.shape[1] != self.input_size:
      raise UnexpectedInputsShape(f'Inputs shape must be (n, {self.input_size}). found {inputs.shape}')

    results = []
    for inp in inputs:
      results.append(self.forward(inp))

    return np.array(results)



  #
  # Backward Pass (loss & optimize)
  #

  def backward(self, loss):

    # gradient
    self.delta = loss * self.activation.apply_derivative(self.output)
    
    # gradients for weights and bias
    self.weight_gradient = self.delta * self.input
    self.bias_gradient = self.delta

    # return loss for previous neurons
    return np.dot(self.delta, self.weights)


  def update(self, learning_rate=None):
    lr = learning_rate or self.learning_rate
    self.weights -= self.weight_gradient
    self.bias    -= self.bias_gradient


  #
  # Train
  #

  def train(self, inputs:Iterable, outputs:Iterable, epochs:int=100, verbose:int=1):
    inputs = np.array(inputs)
    outputs = np.array(outputs)

    if len(inputs.shape) < 2 or inputs.shape[1] != self.input_size:    raise UnexpectedInputsShape (f'Inputs shape must be (n, {self.input_size}). found {inputs.shape}')
    if len(outputs.shape) != 1 or outputs.shape[0] != inputs.shape[0]: raise UnexpectedOutputsShape(f'Outputs size ({outputs.shape[0]}) differs from no of samples in input ({inputs.shape[0]})')
    
    for e in range(epochs):
      
      avg_loss = 0
      
      for i in range(outputs.shape[0]):

        # Forward Pass
        predicted = self.forward(inputs[i])

        # Loss Calculation
        loss = self.loss.calculate(outputs[i], predicted)

        # Backward Pass
        self.backward(loss)

        # Weights Updation
        self.optimizer.update(self)

        if verbose > 1: print(f'epoch={e+1}, sample={i+1}, loss={loss}')
        avg_loss+=loss

      avg_loss /= inputs.shape[0]
      avg_loss = round(avg_loss, 2)
      
      if verbose: print(f'epoch={e+1}, avg_loss={avg_loss}')



  #
  # Test
  #

  def test(self, inputs:Iterable, outputs:Iterable, verbose:int=0):
    inputs = np.array(inputs)
    outputs = np.array(outputs)

    if len(inputs.shape) < 2 or inputs.shape[1] != self.input_size:    raise UnexpectedInputsShape (f'Inputs shape must be (n, {self.input_size}). found {inputs.shape}')
    if len(outputs.shape) != 1 or outputs.shape[0] != inputs.shape[0]: raise UnexpectedOutputsShape(f'Outputs size ({outputs.shape[0]}) differs from no of samples in input ({inputs.shape[0]})')
    

    accurate = 0
    wrong    = 0
    for i in range(inputs.shape[0]):

      # Forward Pass
      predicted = self.forward(inputs[i])

      if predicted == outputs[i]: accurate+=1
      else:                       wrong   +=1

      if verbose > 1: print(f'sample={i+1}, predicted={predicted}, actual={outputs[i]}, error={outputs[i] - predicted}')

    return {
      'total':    accurate + wrong,
      'accurate': accurate,
      'wrong':    wrong,
    }


## Usage

In [85]:
# load dataset
from dataset import X, Y
X.shape

(10, 3)

#### Minimal

In [86]:
neuron = Neuron(3)

In [87]:
neuron.train(X, Y)

epoch=1, avg_loss=0.46
epoch=2, avg_loss=0.5
epoch=3, avg_loss=0.5
epoch=4, avg_loss=0.5
epoch=5, avg_loss=0.5
epoch=6, avg_loss=0.5
epoch=7, avg_loss=0.5
epoch=8, avg_loss=0.5
epoch=9, avg_loss=0.5
epoch=10, avg_loss=0.5
epoch=11, avg_loss=0.5
epoch=12, avg_loss=0.5
epoch=13, avg_loss=0.5
epoch=14, avg_loss=0.5
epoch=15, avg_loss=0.5
epoch=16, avg_loss=0.5
epoch=17, avg_loss=0.5
epoch=18, avg_loss=0.5
epoch=19, avg_loss=0.5
epoch=20, avg_loss=0.5
epoch=21, avg_loss=0.5
epoch=22, avg_loss=0.5
epoch=23, avg_loss=0.5
epoch=24, avg_loss=0.5
epoch=25, avg_loss=0.5
epoch=26, avg_loss=0.5
epoch=27, avg_loss=0.5
epoch=28, avg_loss=0.5
epoch=29, avg_loss=0.5
epoch=30, avg_loss=0.5
epoch=31, avg_loss=0.5
epoch=32, avg_loss=0.5
epoch=33, avg_loss=0.5
epoch=34, avg_loss=0.5
epoch=35, avg_loss=0.5
epoch=36, avg_loss=0.5
epoch=37, avg_loss=0.5
epoch=38, avg_loss=0.5
epoch=39, avg_loss=0.5
epoch=40, avg_loss=0.5
epoch=41, avg_loss=0.5
epoch=42, avg_loss=0.5
epoch=43, avg_loss=0.5
epoch=44, avg_loss=

/tmp/ipykernel_81845/1000496943.py:8: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


In [88]:
neuron.test(X, Y)

/tmp/ipykernel_81845/1000496943.py:8: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


{'total': 10, 'accurate': 5, 'wrong': 5}